In [36]:
import pandas as pd
import numpy as np



In [37]:
data = pd.read_csv('cumulated_data_bl_final_version.csv', delimiter=',')
data['Betting_Prediction'] = ''
data.head()
# pd.read_csv('data_with_betting_odds_v1.csv', delimiter=';')

,Unnamed: 0.1,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Betting_Prediction
0,0,0,D1,10/08/07,Stuttgart,Schalke 04,2,2,D,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,1,1,D1,11/08/07,Bayern Munich,Hansa Rostock,3,0,H,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,2,2,D1,11/08/07,Bochum,Werder Bremen,2,2,D,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,3,3,D1,11/08/07,Ein Frankfurt,Hertha,1,0,H,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,4,4,D1,11/08/07,Hannover,Hamburg,0,1,A,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [38]:
betting_providers = ['B365H','B365D','B365A','BWH','BWD','BWA','IWH','IWD','IWA','VCH','VCD','VCA']
# 'GBH','GBD','GBA', 'WHH','WHD','WHA', 'SBH','SBD', 'SBA', 'SJH','SJD','SJA', 'LBH','LBD','LBA',


In [39]:
def odds_to_probaility_1(odds):
    if odds >= 0:
        probability = odds/(odds+100)
    else:
        probability = odds/(odds-100)
    return probability
print(odds_to_probaility_1(data[betting_providers[1]][0]))

0.031476997578692496


In [40]:
def odds_to_probaility_2(odds):
    probability = (1/odds)*100
    return probability

print(odds_to_probaility_2(data[betting_providers[1]][0]))

30.76923076923077


In [41]:
data[betting_providers[1]][0]

3.25

In [42]:
x,z = 0,0
prob_spieltag, prob_ges = [], []
while x < len(data):
    while z < len(betting_providers):
        prob = odds_to_probaility_1(data[betting_providers[z]][x])
        prob_spieltag.append(prob)
        z +=1
    prob_ges.append(prob_spieltag)
    prob_spieltag = []
    z=0
    x+=1


In [43]:
def calculate_probs_gameday(data):
    x, z = 0, 0
    prob_spieltag, prob_ges = [], []
    while x < len(data):
        while z < len(betting_providers):
            prob = odds_to_probaility_2(data[betting_providers[z]][x])
            prob_spieltag.append(prob)
            z += 1
        prob_ges.append(prob_spieltag)
        prob_spieltag = []
        z = 0
        x += 1
    return prob_ges


In [44]:
prob_ges

[[0.023151313861482857,
  0.031476997578692496,
  0.027899290366481967,
  0.022482893450635383,
  0.031007751937984496,
  0.027710257656781725,
  0.022482893450635383,
  0.03006789524733269,
  0.027237354085603113,
  0.022004889975550123,
  0.031476997578692496,
  0.027237354085603113],
 [0.011466982997232106,
  0.05660377358490566,
  0.15966386554621848,
  0.011369253583786454,
  0.06323185011709602,
  0.1091314031180401,
  0.011857707509881422,
  0.049429657794676805,
  0.0990990990990991,
  0.011271504844769626,
  0.05660377358490566,
  0.11504424778761062],
 [0.033816425120772944,
  0.031945788964181994,
  0.020088192062714356,
  0.03567984570877531,
  0.031476997578692496,
  0.018645731108930322,
  0.031007751937984496,
  0.03006789524733269,
  0.020568070519098924,
  0.033816425120772944,
  0.031476997578692496,
  0.018645731108930322],
 [0.023151313861482857,
  0.031007751937984496,
  0.028182701652089404,
  0.022482893450635383,
  0.031007751937984496,
  0.027710257656781725,
 

In [45]:
len(data)
len(prob_ges)
# len(prob_ges_schnitt)

4590

In [46]:
z, home, draw, away = 0, 0, 0, 0
prob_spieltag_schnitt, prob_ges_schnitt = [], []
for x in prob_ges:
    while z < len(x):
        if z % 3 == 0:
            home = home + x[z]
        if z % 3 == 1:
            draw = draw + x[z]
        if z % 3 == 2:
            away = away + x[z]
        z += 1
    z = 0
    prob_spieltag_schnitt.append(home/4)
    prob_spieltag_schnitt.append(draw/4)
    prob_spieltag_schnitt.append(away/4)
    # print(len(prob_spieltag_schnitt))
    prob_ges_schnitt.append(prob_spieltag_schnitt)
    prob_spieltag_schnitt = []
    # print(len(prob_ges_schnitt))
    # print(home/4)
    # print('')
    home, draw, away = 0, 0, 0


In [47]:
def calculate_average_prob_gameday(prob_ges):
    z, home, draw, away = 0, 0, 0, 0
    prob_spieltag_schnitt, prob_ges_schnitt = [], []
    for x in prob_ges:
        while z < len(x):
            if z % 3 == 0:
                home = home + x[z]
            if z % 3 == 1:
                draw = draw + x[z]
            if z % 3 == 2:
                away = away + x[z]
            z += 1
        z = 0
        prob_spieltag_schnitt.append(home / 4)
        prob_spieltag_schnitt.append(draw / 4)
        prob_spieltag_schnitt.append(away / 4)
        # print(len(prob_spieltag_schnitt))
        prob_ges_schnitt.append(prob_spieltag_schnitt)
        prob_spieltag_schnitt = []
        # print(len(prob_ges_schnitt))
        # print(home/4)
        # print('')
        home, draw, away = 0, 0, 0
    return prob_ges_schnitt


In [48]:
prob_ges_schnitt

[[0.022530497684575938, 0.031007410585675542, 0.02752106404861748],
 [0.011491362233917403, 0.05646726377039604, 0.12073465388774207],
 [0.03358011197207643, 0.03124191984222492, 0.01948693119991848],
 [0.02217176122137594, 0.030772787765321545, 0.028300468638298385],
 [0.02438977960229164, 0.03018520632626048, 0.02593410425172219],
 [0.014972376221724294, 0.03614446638343498, 0.053239824052664975],
 [nan, nan, nan],
 [0.015045154342098336, 0.03579558049238041, 0.053239824052664975],
 [0.01756033951932789, 0.03171127970763262, 0.04006466079877274],
 [0.022410879887361596, 0.03124191984222492, 0.027519684311371778],
 [0.020568070519098924, 0.031124608432047916, 0.031005932268404465],
 [0.022720612397600638, 0.031124608432047916, 0.027352359247480883],
 [0.024865107217332198, 0.03135957260471087, 0.024507768430770915],
 [0.027944986700185248, 0.031124608432047916, 0.022124273915878415],
 [0.017802169828581026, 0.03206219318187718, 0.039140260506439005],
 [0.03322185176048227, 0.031826776

In [49]:
erg_odds = []
for x in prob_ges_schnitt:
    max_value = max(x)
    if x.index(max_value) == 0:
        erg_odds.append('H')
    if x.index(max_value) == 1:
        erg_odds.append('D')
    if x.index(max_value) == 2:
        erg_odds.append('A')
erg_odds

['D',
 'A',
 'H',
 'D',
 'D',
 'A',
 'H',
 'A',
 'A',
 'D',
 'D',
 'D',
 'D',
 'D',
 'A',
 'H',
 'D',
 'A',
 'D',
 'A',
 'D',
 'A',
 'A',
 'D',
 'A',
 'A',
 'D',
 'A',
 'D',
 'D',
 'A',
 'D',
 'D',
 'A',
 'H',
 'H',
 'D',
 'A',
 'A',
 'D',
 'A',
 'A',
 'A',
 'D',
 'A',
 'D',
 'D',
 'A',
 'D',
 'D',
 'A',
 'A',
 'D',
 'H',
 'D',
 'H',
 'A',
 'D',
 'A',
 'D',
 'A',
 'D',
 'A',
 'A',
 'D',
 'A',
 'H',
 'D',
 'H',
 'A',
 'D',
 'A',
 'A',
 'D',
 'H',
 'A',
 'A',
 'A',
 'A',
 'A',
 'D',
 'A',
 'H',
 'A',
 'H',
 'A',
 'D',
 'A',
 'D',
 'A',
 'D',
 'A',
 'D',
 'A',
 'D',
 'D',
 'D',
 'H',
 'H',
 'H',
 'A',
 'A',
 'D',
 'A',
 'D',
 'A',
 'D',
 'A',
 'D',
 'A',
 'A',
 'D',
 'D',
 'H',
 'A',
 'D',
 'D',
 'D',
 'A',
 'A',
 'H',
 'D',
 'D',
 'A',
 'A',
 'D',
 'D',
 'D',
 'D',
 'A',
 'A',
 'D',
 'A',
 'H',
 'D',
 'A',
 'A',
 'D',
 'H',
 'A',
 'D',
 'A',
 'A',
 'D',
 'D',
 'H',
 'H',
 'D',
 'A',
 'A',
 'D',
 'D',
 'D',
 'H',
 'D',
 'H',
 'D',
 'A',
 'A',
 'D',
 'A',
 'A',
 'A',
 'A',
 'A',
 'D',
 'A'

In [50]:
def get_odd_prediction(prob_ges_schnitt):
    erg_odds, erg_prob = [], []
    for x in prob_ges_schnitt:
        max_value = max(x)
        erg_prob.append(max_value)
        if x.index(max_value) == 0:
            erg_odds.append('H')
        if x.index(max_value) == 1:
            erg_odds.append('D')
        if x.index(max_value) == 2:
            erg_odds.append('A')
    return erg_odds, erg_prob

In [51]:
data['Betting_Prediction'] = erg_odds

In [52]:
def add_odd_predcition_to_df(data, erg_odds, erg_prob):
    data['Betting_Prediction'] = erg_odds
    data['Betting_Prediction_Probability'] = erg_prob
    return data

In [53]:
def df_to_csv(data):
    data.to_csv("../dataframe_with_odd_predictions.csv")

In [54]:
data.head()

,Unnamed: 0.1,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Betting_Prediction
0,0,0,D1,10/08/07,Stuttgart,Schalke 04,2,2,D,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D
1,1,1,D1,11/08/07,Bayern Munich,Hansa Rostock,3,0,H,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
2,2,2,D1,11/08/07,Bochum,Werder Bremen,2,2,D,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H
3,3,3,D1,11/08/07,Ein Frankfurt,Hertha,1,0,H,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D
4,4,4,D1,11/08/07,Hannover,Hamburg,0,1,A,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D


In [55]:
data.to_csv("../dataframe_with_odd_predictions_v2.csv")

In [56]:
z, right_prediction, false_prediction = 0,0, 0
while z < len(data):
    if data.FTR[z] == data.Betting_Prediction[z]:
        right_prediction +=1
    else:
        false_prediction +=1
    z+=1

In [57]:
def evaluate_odd_performance(data):
    z, right_prediction, false_prediction = 0, 0, 0
    while z < len(data):
        if data.FTR[z] == data.Betting_Prediction[z]:
            right_prediction += 1
        else:
            false_prediction += 1
        z += 1
    return right_prediction, false_prediction

In [58]:
print('Right Predictions: ',right_prediction)
print('False Predictions: ',false_prediction)
print('Number of Games: ',len(data))

Right Predictions:  1033
False Predictions:  3557
Number of Games:  4590


In [59]:
def show_odd_performance(data, right_prediction, false_prediction):
    print('Right Predictions: ', right_prediction)
    print('False Predictions: ', false_prediction)
    print('Number of Games: ', len(data))
    print('Accuracy: ', round(right_prediction/len(data)*100,2), '%')


In [60]:
print(data.B365H[0])

2.37


In [61]:
def run_all(data):
    prob_ges = calculate_probs_gameday(data)
    prob_ges_schnitt = calculate_average_prob_gameday(prob_ges)
    erg_odds, erg_prob = get_odd_prediction(prob_ges_schnitt)
    data = add_odd_predcition_to_df(data, erg_odds, erg_prob)
    right_prediction, false_prediction = evaluate_odd_performance(data)
    show_odd_performance(data, right_prediction, false_prediction)
run_all(data)

Right Predictions:  2366
False Predictions:  2224
Number of Games:  4590
Accuracy:  51.55 %


In [62]:
data.head()

,Unnamed: 0.1,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Betting_Prediction,Betting_Prediction_Probability
0,0,0,D1,10/08/07,Stuttgart,Schalke 04,2,2,D,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,43.398765
1,1,1,D1,11/08/07,Bayern Munich,Hansa Rostock,3,0,H,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,86.054012
2,2,2,D1,11/08/07,Bochum,Werder Bremen,2,2,D,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,50.415673
3,3,3,D1,11/08/07,Ein Frankfurt,Hertha,1,0,H,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,44.145361
4,4,4,D1,11/08/07,Hannover,Hamburg,0,1,A,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H,40.032051


In [63]:
max_value_home = []
max_value_away = []
max_value_draw = []
for index, rows in data.iterrows():
    max_value_home = max(rows['B365H'], rows['BWH'], rows["IWH"], rows['VCH'])
    max_value_away = max(rows['B365A'], rows['BWA'], rows["IWA"], rows['VCA'])
    max_value_draw = max(rows['B365D'], rows['BWD'], rows["IWD"], rows['VCD'])
data['MaxHome'] = max_value_home
data['MaxAway'] = max_value_away
data['MaxDraw'] = max_value_draw

In [64]:
data.to_csv("./dataframe_with_odd_predictions_v2.csv")